In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

Read Raw datasets: sheet 0 stores the surgical and demographical data, as well as PROMs scores; sheet 1 contains PROMs question items

In [ ]:
anca =  pd.read_excel("estrazione_datareg_20230803.xlsx", sheet_name=0)
anca_risposte = pd.read_excel("estrazione_datareg_20230803.xlsx", sheet_name=1)
anca_risposte = anca_risposte[anca_risposte['cartella_clinica'].notna()]
ginocchio = pd.read_excel("estrazione_datareg_20230803.xlsx", sheet_name=2)
ginocchio_risposte = pd.read_excel("estrazione_datareg_20230803.xlsx", sheet_name=3)
ginocchio_risposte = ginocchio_risposte[ginocchio_risposte['cartella_clinica'].notna()]
spine = pd.read_excel("spinereg.xlsx", sheet_name=0, dtype=dict_types)
spine_risposte = pd.read_excel("spinereg.xlsx", sheet_name=1, dtype=dict_types)

Drop identifying keys

In [ ]:
ginocchio_riposte = ginocchio_risposte.drop(columns=["cartella_clinica", "MSP", "id_episodio_esterno"])
anca_risposte = anca_risposte.drop(columns=["cartella_clinica", "MSP", "id_episodio_esterno"])
spine_risposte = spine_risposte.drop(columns=["cartella_clinica", "MSP", "id_episodio_esterno"])

(Left-)join datasets based on id (surgical event id). Drop uid (patient id) from second table. Subsequently, remove patients who withdrew from register.

In [ ]:
hip_join = hip.set_index("id").join(hip_risposte.set_index("id").drop(columns="uid"))
hip_join = hip_join[(hip_join["state"] != "paziente uscito dal registro") & (hip_join["state"] != "paziente che ha rinunciato")]

knee_join = knee.set_index("id").join(knee_risposte.set_index("id").drop(columns="uid"))
knee_join = knee_join[(knee_join["state"] != "paziente uscito dal registro") & (knee_join["state"] != "paziente che ha rinunciato")]

spine_join = spine.set_index("id").join(spine_risposte.set_index("id").drop(columns="uid"))
spine_join = spine_join[(spine_join["state"] != "paziente uscito dal registro") & (spine_join["state"] != "paziente che ha rinunciato")]

Hash operator names using SHAKE-128

In [ ]:
import hashlib

def hash_fun(x):
    try:
        hashGen = hashlib.shake_128()
        hashGen.update(x.encode('utf-8'))
        return hashGen.hexdigest(10)
    except:
        return x

spine_join["first_operator_1"] = spine_join["first_operator_1"].apply(lambda x: hash_fun(x))
spine_join["first_operator_2"] = spine_join["first_operator_2"].apply(lambda x: hash_fun(x))
spine_join["first_operator_3"] = spine_join["first_operator_3"].apply(lambda x: hash_fun(x))

Translate column names for questionnaire items in SpineReg Spine registry. The remaining columns have been translated by directly operating on the .csv files.

In [ ]:
translate = {
    #BMI
    'peso' : "weight",
    'altezza' : "height",

    #ODI
    'ODI_dolore' : "ODI_pain_intensity",
    'curapersonale' : "personal_care",
    'alzarepesi' : "lifting",
    'camminare' : "walking",
    'stareseduto': "sitting",
    'stareinpiedi': "standing",
    'dormire' : "sleeping",
    'vitasessuale' : "sex_life",
    'vitasociale' : "social_life",
    'viaggiare' : "travelling",

    #COMI
    'assiale' : "axial",
    'radicolare': "radicular",
 
    #SF36
    'salutegenerale' : "health_self_evaluation",
    'saluteannofa' : "health_compared_year_ago",
    'attimpegnative' : "vigorous_activity",
    'attmoderate': "moderate_activity",
    'portareborsespesa' : "carrying_groceries",
    'salirequalchepiano' : "climbing_several_stairs",
    'salireunpiano' : "climbing_stairs",
    'piegarsichinarsi': "bending_kneeling",
    'cammkm' : "walk_more_than_mile",
    'cammcentinaiam' : "walk_several_blocks",
    'cammcentom' : "walk_block",
    'farebagno' : "bathing_dressing",
    'ridottotempolav': "cut_time_work",
    'resomeno': "accomplished_less_physical",
    'limittipilav': "limited_in_activities",
    'difficoltalav' : "difficulties_work",
    'ridottotempolavem' : "cut_time_work_emotional",
    'resomenoem': "accomplished_less_emotional",
    'caloconcem' : "less_careful",
    'interveritoatt' : "social_activities_interference_extent",
    'dolorefisico': "bodily_pain",
    'doloreostacolato': "pain_interference",
    'vivacebrillante': "full_of_pep",
    'moltoagitato' : "nervous",  
    'giumorale': "felt_down",
    'calmosereno': "calm_peaceful",
    'pienoenergia': "energy",
    'scoraggtriste': "downhearted_blue",
    'felice': "happy",
    'sfinito': "worn_out",
    'felice': "happy",
    'stanco': "tired",
    'saluteinterferito':  "social_activities_interference_time",
    'saluteammalato': "sick_more_easily",
    'salutecomealtri': "healthy_as_others",
    'salutepeggiora': "health_worsening",
    'saluteottima': "health_excellent",
    
    
    #SRS22
    "terminedolseimesi" : "pain_last_six_months",
    "terminedolultimomese": "pain_last_month",
    "seimesinervoso": "nervous_last_six_months",
    "aspettorestovita": "life_current_back_shape",
    "attquotnormalm": "current_activity_level",
    "aspettovestito": "look_in_clothes",
    "seimesidepresso": "depressed_last_six_months",
    "maldischienarisposo": "pain_at_rest",
    "livelloatt": "level_activity",
    "terminiaspettotronco": "appearance_trunk",
    "farmacischiena": "pain_medication",
    "schienalimatt": "limit_house_activities",
    "seimesicalmo": "calm_peaceful",
    "schienarelpers": "affect_personal_relationship",
    "difficoltaeconomiche": "financial_difficulties",
    "seimesidisperato": "downhearted_blue",
    "tremesiassenze": "days_off",
    "uscire": "limit_going_out",
    "schienaattraente": "feel_attractive",
    "seimesifelice": "happy",
    "soddisftrattam": "satisfaction",
    "ancoratrattam": "repeat_same_management",
    
    #FABQ
    "attfisprovoca": "physical_activity_causes_pain",
    "attfisaggrava": "physical_activity_worsens_pain",
    "attfisdannschiena": "physical_activity_may_harm",
    "nondovreiattfis": "physical_activity_should_not_perform",
    "nonriescoattfis": "physical_activity_cannot_perform",
    "lavprovoca": "work_causes_pain",
    "lavaggravato": "work_worsens_pain",
    "richiestoindenn": "claimed_compensation",
    "lavpesante": "work_too_heavy",
    "lavaggrava": "work_would_worsen_pain",
    "lavdannschiena": "work_might_harm",
    "nondovreilav": "should_not_work",
    "nonriuscirolav": "cannot_work",
    "nonlavtremesi": "cannot_work_three_months",
    "nonlavmai": "ever_back_work",
    "attualmlav": "currently_working",
    "motivononlav": "reason_not_work"
}

In [ ]:
import re

new_cols = []
for c in spine_join.columns:
    check = True
    for k in translate.keys():
        if k in c:
            print(c)
            new_cols.append(re.sub(k, translate[k], c))
            print(new_cols[-1])
            print()
            check = False
            break
    if check:
        new_cols.append(c)

spine_join.columns = new_cols

Transform binary columns in 0/1 format

In [ ]:
spine_join = spine_join.replace("si", 1)
spine_join = spine_join.replace("no", 0)

spine_join["sex"] = spine_join["sex"].apply(lambda x: 1 if x == "F" else 0 if x == "M" else np.nan)

new_cols = []
for c in spine_join.columns:
    if c == "sex":
        new_cols.append("sex_0M_1F")
    else:
        new_cols.append(c)
        
spine_join.columns = new_cols

Discard patients who were not of age at the time of enrollment

In [ ]:
hip_join = hip_join[hip_join["age_at_enrollment"] >= 18]
knee_join = knee_join[knee_join["age_at_enrollment"] >= 18]
spine_join = spine_join[spine_join["age_at_enrollment"] >= 18]

Validate BMI values and set to NaN values that are found to be invalid ($|\frac{Weight}{(Height/100)^2} - BMI| > 1$), to denote missing information

In [ ]:
months = ["PreOp", "3months", "6months", "12months", "24months", "60months"]
for m in months:
    cond = np.abs(hip_join["BMI_weight_"+m]/(hip_join["BMI_height_"+m]/100)**2 - hip_join["BMI_Total_"+m]) > 1
    tmp = hip_join[cond]
    if len(tmp) > 0:
        hip.loc[cond, "BMI_Total_"+m] = 0

months = ["PreOp", "3months", "6months", "12months", "24months", "60months"]
for m in months:
    cond = np.abs(knee_join["BMI_weight_"+m]/(knee_join["BMI_height_"+m]/100)**2 - knee_join["BMI_Total_"+m]) > 1
    tmp = knee_join[cond]
    if len(tmp) > 0:
        knee_join.loc[cond, "BMI_Total_"+m] = 0

for m in months:
    cond = np.abs(spine_join["BMI_weight_ans_0"]/(spine_join["BMI_height_ans_0"]/100)**2 - spine_join["BMI_Total_PreOp"]) > 1
    tmp = spine_join[cond]
    if len(tmp) > 0:
        spine_join.loc[cond, "BMI_Total_PreOp"] = 0

Extract spine level from surgical procedure

In [ ]:
levels = spine_join["surgical_procedure_1"].str.extractall(r'([lscdtLSCDT][0-9]+-?[lscdtLSCDT][0-9]+)|(ileo)|(ILEO)|(LOMB[A-Z]*)|(lomb[a-z]*)|(sacr[a-z]*)|(SACR[A-Z]*)|( I *[LDCS] )|( II *[LDCS] )|( III *[LDCS] )|( IV *[LDCS] )|( V *[LDCS] )')
spine_join["levels_1"] = levels.fillna("").agg(" ".join, axis=1).str.strip().groupby(level=0).agg(" ".join)


levels = spine_join["surgical_procedure_2"].str.extractall(r'([lscdtLSCDT][0-9]+-?[lscdtLSCDT][0-9]+)|(ileo)|(ILEO)|(LOMB[A-Z]*)|(lomb[a-z]*)|(sacr[a-z]*)|(SACR[A-Z]*)|( I *[LDCS] )|( II *[LDCS] )|( III *[LDCS] )|( IV *[LDCS] )|( V *[LDCS] )')
spine_join["levels_2"] = levels.fillna("").agg(" ".join, axis=1).str.strip().groupby(level=0).agg(" ".join)

levels = spine_join["surgical_procedure_3"].str.extractall(r'([lscdtLSCDT][0-9]+-?[lscdtLSCDT][0-9]+)|(ileo)|(ILEO)|(LOMB[A-Z]*)|(lomb[a-z]*)|(sacr[a-z]*)|(SACR[A-Z]*)|( I *[LDCS] )|( II *[LDCS] )|( III *[LDCS] )|( IV *[LDCS] )|( V *[LDCS] )')
spine_join["levels_3"] = levels.fillna("").agg(" ".join, axis=1).str.strip().groupby(level=0).agg(" ".join)

Translate variables' texts from English to Italian

In [ ]:
def change_anesthesia(x):
    if x == "Anestesia generale":
        return "General"
    elif x == "Locale eseguita dal 1Operatore":
        return "Local"
    elif x == "Subaracnoidea":
        return "Subarachnoid"
    elif x == "Altro":
        return "Other"
    else:
        return x

spine_join["anesthesia_1"] = spine_join["anesthesia_1"].apply(change_anesthesia)
spine_join["anesthesia_2"] = spine_join["anesthesia_2"].apply(change_anesthesia)
spine_join["anesthesia_3"] = spine_join["anesthesia_3"].apply(change_anesthesia)

In [ ]:
def change_side(x):
    if x == "Sinistra":
        return "Left"
    elif x == "Destra":
        return "Right"
    elif x == "Bilaterale":
        return "Bilateral"
    elif x == "Non Applicabile":
        return "Not Applicable"
    else:
        return x

spine_join["intervention_side_1"] = spine_join["intervention_side_1"].apply(change_side)
spine_join["intervention_side_2"] = spine_join["intervention_side_2"].apply(change_side)
spine_join["intervention_side_3"] = spine_join["intervention_side_3"].apply(change_side)

In [ ]:
def change_clean(x):
    if x == "Pulito":
        return "Clean"
    elif x == "Sporco":
        return "Unclean"
    elif x == "Contaminato":
        return "Contaminated"
    else:
        return x

spine_join["clean_intervention_1"] = spine_join["clean_intervention_1"].apply(change_clean)
spine_join["clean_intervention_2"] = spine_join["clean_intervention_2"].apply(change_clean)
spine_join["clean_intervention_3"] = spine_join["clean_intervention_3"].apply(change_clean)

In [ ]:
def change_category(x):
    if x == "Artrodesi lombare":
        return "Lumbar arthrodesis"
    elif x == "Ernia lombare":
        return "Lumbar herniation"
    elif x == "Deformita degenerativa":
        return "Degenerative deformity"
    elif x == "Decompressione lombare":
        return "Lumbar decompression"
    elif x == "Artrodesi cervicale":
        return "Cervical arthrodesis"
    elif x == "Deformita idiopatica":
        return "Idiopatic deformity"
    elif x == "Cifoplastiche":
        return "Kyphoplasty"
    elif x == "Tumore vertebrale":
        return "Vertebral cancer"
    elif x == "Ernia cervicale":
        return "Cervical herniation"
    elif x == "Scoliosi neuromuscolare ridotto":
        return "Neuromuscular scoliosis"
    else:
        return x

spine_join["category"] = spine_join["category"].apply(change_category)

In [ ]:
def change_state(x):
    if x == "attivo":
        return "active"
    elif x == "chiuso per re-intervento fallimento":
        return "close (reintervention after failure)"
    elif x == "chiuso per re-intervento con diverso protocollo":
        return "close (reintervention with different protocol)"
    elif x == "chiuso per re-intervento revisione o programmato":
        return "close (planned reintervention or revision)"
    elif x == "chiuso per morte":
        return "close (death)"
    elif x == "paziente rioperato in altra struttura":
        return "patient moved to other structure"
    elif x == "chiuso per mancata comunicazione":
        return "close (lack of communications)"
    elif x == "chiusura per completa mancanza di compilazione dei formulari":
        return "close (questionnaires completely unfilled)"
    elif x == "chiusura per paziente con problemi di compilazione":
        return "close (patient with compilation problems)"
    elif x == "chiuso per re-intervento per infezione o sospetta infezione":
        return "close (infection or suspected infection)"
    elif x == "chiuso per re-intervento":
        return "close (reintervention)"
    elif x == "chiuso":
        return "close"
    elif x == "paziente non operato":
        return "not operated"
    else:
        return x

spine_join["state"] = spine_join["state"].apply(change_state)
hip_join["state"] = hip_join["state"].apply(change_state)
knee_join["state"] = knee_join["state"].apply(change_state)

In [ ]:
def change_patient(x):
    if x == "paziente_attivo":
        return "active"
    elif x == "paziente_inattivo":
        return "inactive"
    else:
        return x

spine_join["patient_state"] = spine_join["patient_state"].apply(change_patient)
hip_join["patient_state"] = hip_join["patient_state"].apply(change_patient)
knee_join["patient_state"] = knee_join["patient_state"].apply(change_patient)

In [ ]:
def change_step(x):
    if x == "Intraoperatorio-Dimissione":
        return "Intraop-Discharge"
    elif x == "3 mesi":
        return "3 months"
    elif x == "6 mesi":
        return "6 months"
    elif x == "12 mesi":
        return "12 months"
    elif x == "24 mesi":
        return "24 months"
    elif x == "5 anni":
        return "5 years"
    elif x == "10 anni":
        return "10 years"
    else:
        return x

spine_join["current_step"] = spine_join["current_step"].apply(change_step)
hip_join["current_step"] = hip_join["current_step"].apply(change_step)
knee_join["current_step"] = knee_join["current_step"].apply(change_step)

In [ ]:
def change_category(x):
    if x == "Anca":
        return "First intervention"
    elif x == "Anca revisione":
        return "Revision"
    elif x == "Anca bilaterale":
        return "Bilateral intervention"
    else:
        return x

hip_join["category"] = hip_join["category"].apply(change_category)

In [ ]:
def change_side(x):
    if x == "SINISTRA":
        return "Left"
    elif x == "DESTRA":
        return "Right"
    elif x == "BILATERALE":
        return "Bilateral"
    else:
        return x

hip_join["side"] = hip_join["event"].str.extract(r'(SINISTRA)|(DESTRA)|(BILATERALE)').fillna("").agg(" ".join, axis=1).str.strip().apply(change_side)
hip_join["side"]

In [ ]:
def change_category(x):
    if x == "Ginocchio":
        return "First intervention"
    elif x == "Ginocchio revisione":
        return "Revision"
    elif x == "Ginocchio bilaterale":
        return "Bilateral intervention"
    else:
        return x

knee_join["category"] = knee_join["category"].apply(change_category)
knee_join["category"].value_counts()

In [ ]:
def change_side(x):
    if x == "SINISTRO":
        return "Left"
    elif x == "DESTRO":
        return "Right"
    elif x == "BILATERALE":
        return "Bilateral"
    else:
        return x

knee_join["side"] = knee_join["event"].str.extract(r'(SINISTRO)|(DESTRO)|(BILATERALE)').fillna("").agg(" ".join, axis=1).str.strip().apply(change_side)
knee_join["side"].value_counts()

Drop potentially identifying columns and save to publicly released datasets

In [ ]:
hip = hip.drop(columns=["event", "surgical_procedure", "equipe"])
hip.to_csv("OGSA_PROMs_2023_hip.csv", index=False)

knee = knee.drop(columns=["event", "surgical_procedure", "equipe"])
knee.to_csv("OGSA_PROMs_2023_knee.csv", index=False)

spine_join = spine_join.drop(columns=["equipe_1", "unit_1", "equipe_2", "unit_2", "equipe_3", "unit_3",
                                      "event", "surgical_procedure_1", "surgical_procedure_2", "surgical_procedure_3"])
spine_join.to_csv("OGSA_PROMs_2023_spine.csv", index=False)

# Validation

In [14]:
dict_types = {
 'ICD': object,
 'ICD_1': object,
 'ICD_code_PREOP_1': object,
 'ICD_name_PREOP_1': object,
 'ICD_code_INTRAOP_1': object,
 'ICD_name_INTRAOP_1': object,
 'ICD_code_POSTOP_1': object,
 'ICD_name_POSTOP_1': object,
 'ICD_2': object,
 'ICD_code_PREOP_2': object,
 'ICD_name_PREOP_2': object,
 'ICD_code_INTRAOP_2': object,
 'ICD_name_INTRAOP_2': object,
 'ICD_code_POSTOP_2': object,
 'ICD_name_POSTOP_2': object,
 'ICD_3': object,
 'ICD_code_PREOP_3': object,
 'ICD_name_PREOP_3': object,
 'ICD_code_INTRAOP_3': object,
 'ICD_name_INTRAOP_3': object,
 'ICD_code_POSTOP_3': object,
 'ICD_name_POSTOP_3': object
}

In [15]:
hip = pd.read_csv("OGSA_PROMs_2023_hip.csv", dtype=dict_types)
knee = pd.read_csv("OGSA_PROMs_2023_knee.csv", dtype=dict_types)
spine = pd.read_csv("OGSA_PROMs_2023_spine.csv", dtype=dict_types)

/tmp/ipykernel_361/3732519904.py:3: DtypeWarning: Columns (42,43,44,45,46,48,58,89,219,327,435,500,529,541,543,561,608,649,669,716,747) have mixed types. Specify dtype option on import or set low_memory=False.
  spine = pd.read_csv("OGSA_PROMs_2023_spine.csv", dtype=dict_types)


Check BMI validity

In [17]:
months = ["PreOp", "3months", "6months", "12months", "24months", "60months"]

check = True
for m in months:
    cond = (np.abs(hip["BMI_weight_"+m]/(hip["BMI_height_"+m]/100)**2 - hip["BMI_Total_"+m]) > 1) & (hip["BMI_Total_"+m] != 0)
    tmp = hip[cond]
    if len(tmp) > 0:
        check = False
print(check)

check = True
months = ["PreOp", "3months", "6months", "12months", "24months", "60months"]
for m in months:
    cond = (np.abs(knee["BMI_weight_"+m]/(knee["BMI_height_"+m]/100)**2 - knee["BMI_Total_"+m]) > 1)  & (knee["BMI_Total_"+m] != 0)
    tmp = knee[cond]
    if len(tmp) > 0:
        check = False
print(check)

check = True
for m in months:
    cond = (np.abs(spine["BMI_weight_ans_0"]/(spine["BMI_height_ans_0"]/100)**2 - spine["BMI_Total_PreOp"]) > 1) & (spine["BMI_Total_PreOp"] != 0)
    tmp = spine[cond]
    if len(tmp) > 0:
        check = False
print(check)

True
True
True
